# Understanding CV Metrics and Graphs

In this lesson, we will dive deep into the essential **metrics and graphs** used to evaluate computer vision models during training and after deployment. Understanding these metrics is crucial for diagnosing issues like overfitting and underfitting, as well as improving model performance. We will cover key metrics such as **accuracy**, **precision**, **mAP (mean Average Precision)**, and discuss important concepts like **training loss** and **validation loss**, and their associated graphs.

---

## Learning Objectives

By the end of this section, you will:
- Understand the importance of key metrics such as **accuracy**, **precision**, **mAP**, and **F1 score** for evaluating model performance.
- Learn how to interpret graphs for **training loss**, **validation loss**, and accuracy.
- Recognize the signs of **overfitting** and **underfitting**, and understand how to address these issues.
- Gain practical knowledge of how these metrics relate to model generalization and performance on unseen data.

---

## Key Metrics in Computer Vision

Computer vision models are evaluated using several metrics that provide insight into how well the model is performing on both training and unseen data. Here, we break down some of the most important metrics used in image classification, object detection, and segmentation tasks.

### Accuracy

**Accuracy** measures the percentage of correct predictions out of the total predictions made by the model.

#### Formula:
\[
\text{Accuracy} = \frac{\text{Correct Predictions}}{\text{Total Predictions}} \times 100
\]

While accuracy is a good metric when classes are balanced, it can be misleading in cases where the dataset is imbalanced. For example, in a marine species detection task, if 95% of the dataset consists of images without fish, a model that predicts "no fish" 100% of the time would still have high accuracy despite being ineffective.

---

### Precision

**Precision** measures how many of the predicted positive instances (e.g., correctly classified images) are actually correct.

#### Formula:
\[
\text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}
\]

Precision is useful in scenarios where false positives are costly, such as in object detection, where incorrectly classifying background elements as objects would reduce precision.

**Use Case**: In a marine species classification task, high precision ensures that when the model predicts the presence of a species (e.g., fish), it is more likely to be correct.

---

### Recall

**Recall** measures how many of the actual positive instances were correctly identified by the model.

#### Formula:
\[
\text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
\]

**Use Case**: In underwater species detection, recall is crucial if it’s important not to miss any instances of a rare species. High recall ensures the model detects as many true positives as possible, even at the cost of a few false positives.

---

### F1 Score

The **F1 Score** is the harmonic mean of precision and recall. It balances the trade-off between the two metrics, especially when dealing with imbalanced datasets.

#### Formula:
\[
F1 = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
\]

**Use Case**: F1 score is particularly useful in marine imagery with imbalanced datasets, such as detecting small or rare species.

---

### Mean Average Precision (mAP)

In object detection tasks, the **mean Average Precision (mAP)** is a widely used metric that evaluates how well the model predicts the bounding boxes for multiple classes.

#### Formula:
mAP is calculated by averaging the **Average Precision (AP)** across all classes:
\[
\text{mAP} = \frac{1}{n} \sum_{i=1}^{n} AP_i
\]

- **AP** measures the area under the precision-recall curve for each class.
- **mAP** is the mean of these values across all classes, providing a single metric to evaluate model performance.

**Use Case**: In a marine object detection task where you are detecting multiple species or objects (e.g., fish, crabs, underwater debris), mAP provides a more comprehensive metric than accuracy alone, as it considers both the precision and recall of bounding boxes.

---

## Training and Validation Loss

### Training Loss

**Training loss** measures how well the model is performing on the training data. It is calculated by comparing the model’s predictions to the actual labels and computing a loss function (e.g., **categorical cross-entropy** for classification tasks).

#### Interpretation:
- **Decreasing Training Loss**: A decreasing training loss indicates that the model is learning from the data.
- **Low Training Loss**: If the training loss is too low while the validation loss is high, the model may be **overfitting** to the training data.

---

### Validation Loss

**Validation loss** measures how well the model is performing on the validation dataset, which is not seen by the model during training. It is crucial for evaluating the model's ability to generalize to new data.

#### Interpretation:
- **Increasing Validation Loss**: If the validation loss increases while the training loss decreases, it suggests the model is overfitting.
- **Balanced Validation Loss**: If the validation loss decreases and converges alongside the training loss, the model is likely generalizing well.

---

### Overfitting and Underfitting

#### Overfitting

**Overfitting** occurs when the model learns too much from the training data, capturing noise and patterns that do not generalize to unseen data. This results in:
- **Low training loss** but **high validation loss**.
- Poor performance on the test dataset.

#### How to Address Overfitting:
- **Early stopping**: Stop training when the validation loss starts increasing.
- **Data augmentation**: Increase the variety of the training data by applying transformations like rotation, flipping, or scaling.
- **Regularization**: Techniques like **dropout** or **L2 regularization** can help prevent overfitting by making the model less complex.

---

#### Underfitting

**Underfitting** occurs when the model is too simple and fails to learn the underlying patterns in the data. This results in:
- **High training loss** and **high validation loss**.
- Poor performance on both training and test datasets.

#### How to Address Underfitting:
- **Increase model complexity**: Add more layers or units to the model.
- **Train for more epochs**: Ensure the model has enough time to learn from the data.
- **Use better features**: Improve the quality or variety of the training data.

---

## Training and Validation Accuracy

### Training Accuracy

**Training accuracy** refers to the model's accuracy on the training dataset. A high training accuracy means that the model is able to correctly classify a large portion of the training data.

#### Interpretation:
- **Increasing Training Accuracy**: A steadily increasing training accuracy is a good sign that the model is learning.
- **Too High Training Accuracy**: If training accuracy becomes very high, but validation accuracy is low, this could indicate **overfitting**.

---

### Validation Accuracy

**Validation accuracy** is the model's accuracy on the validation dataset, which represents its ability to generalize to new, unseen data.

#### Interpretation:
- **Balanced Training and Validation Accuracy**: If training and validation accuracy rise together, this indicates a well-performing model.
- **Validation Accuracy Drops**: If validation accuracy decreases while training accuracy increases, this suggests **overfitting**.

---

## Graphs: Training vs Validation Metrics

### Loss Graphs

- **Training Loss vs Validation Loss**: A typical loss graph should show both training and validation loss decreasing over time. If validation loss starts to increase while training loss decreases, it indicates overfitting.

### Accuracy Graphs

- **Training Accuracy vs Validation Accuracy**: The accuracy graph should ideally show both training and validation accuracy increasing. If the gap between training and validation accuracy becomes too large, the model may be overfitting.

---
